# Applied Deep Learning Tutorial 
# Deep Reinforcement Learning with Deep-Q-Network (DQN)

## Introduction
In this tutorial, you will attempt to implement a Deep-Q-Network that is able to do a classic control. The approaches are build upon the paper by DeepMind: Playing Atari with Deep Reinforcement Learning [paper](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf), which first introduces the notion of a Deep Q-Network.
<img src="graphics/atari_play.png" width="700"><br>
<center> Fig. 1: Breakout environment of the Atari game </center>

## Core idea
As you probably remember from the lecture, during trial and error we can learn a policy for our Atari game, and model it within our Q-matrix. This is done with a deep neural network. After training, this Q-matrix gives us an estimate of the expected reward when taking action a in state s: Q(s, a).
Playing the action with the maximum Q-value in any given state is the same as playing optimal, or following a full exploitation strategy.

## OpenAI Gym
[OpenAI Gym](https://gym.openai.com/docs/) is a library that can simulate a large number of reinforcement learning environments, including Atari games (these need to be installed additionaly). You will need Python 3.5+

>pip install gym


## Taking our cart pole on a first ride
Now that you have gym installed you can load the 'Pendulum-v0' environment of Atari.


In [50]:
# Import the gym module
import gym


In [51]:
# Load the environment
env = gym.make('Pendulum-v0')

# Reset, it returns the starting frame
frame = env.reset()

for _ in range(100000):
    # Perform a random action, returns the new frame, reward and whether the game is over
    action = env.action_space.sample()
    observation, reward, is_done, info = env.step(action)
    print('observation: ', observation, 'reward: ', reward)
    if is_done: break
    env.render()

env.close()


observation:  [ 0.86493428 -0.50188513 -0.55901944] reward:  -0.2527977999118114
observation:  [ 0.83464549 -0.55078753 -1.15061163] reward:  -0.3097494877120603
observation:  [ 0.79602129 -0.60526862 -1.3359158 ] reward:  -0.47494441788282654
observation:  [ 0.73525871 -0.67778656 -1.89289474] reward:  -0.601573143554305
observation:  [ 0.65985849 -0.7513899  -2.10835792] reward:  -0.9167669661274144
observation:  [ 0.55508021 -0.83179683 -2.64342032] reward:  -1.1673354431915164
observation:  [ 0.42471827 -0.90532557 -2.99617455] reward:  -1.6670192404112978
observation:  [ 0.23821938 -0.97121137 -3.96237256] reward:  -2.1831260346424
observation:  [-1.94282752e-03 -9.99998113e-01 -4.84949737e+00] reward:  -3.3407622952310305
observation:  [-0.27963141 -0.96010743 -5.62934702] reward:  -4.82531052925543
observation:  [-0.56235717 -0.82689444 -6.27646994] reward:  -6.607273120874804
observation:  [-0.80020766 -0.59972301 -6.60815458] reward:  -8.643460447705376
observation:  [-0.95592

observation:  [0.99970808 0.02416083 3.36359403] reward:  -1.2898567693292018
observation:  [0.98014441 0.19828498 3.50889325] reward:  -1.1326792024098062
observation:  [0.9327887  0.3604237  3.38228462] reward:  -1.2744454743618234
observation:  [0.84988949 0.52696097 3.72597421] reward:  -1.280180157183985
observation:  [0.72313105 0.69071085 4.14901644] reward:  -1.696370860934642
observation:  [0.53195302 0.84677387 4.94838772] reward:  -2.306314529037839
observation:  [0.27214774 0.96225548 5.70562034] reward:  -3.4691970886978196
observation:  [-0.04112822  0.99915388  6.33528277] reward:  -4.933257309238082
observation:  [-0.38488876  0.92296297  7.07894933] reward:  -6.6119203883280155
observation:  [-0.70704555  0.70716801  7.80448563] reward:  -8.875900099297148
observation:  [-0.9330223   0.35981856  8.        ] reward:  -11.642251971799386
observation:  [-0.99945102 -0.03313097  8.        ] reward:  -14.09509100682671
observation:  [-0.90440625 -0.42667239  8.        ] rew

This already looks nice, yet the actions are random and thus it is time to better understand our environment. And to implement our Deep-Q-Network


In [135]:
# import the necessary libraries
import gym
import gym.spaces
import gym.wrappers
import numpy as np
import random
import pickle
from collections import deque
from keras.layers import Flatten, Dense
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras import optimizers

## Observation
The observation is made up of cos(theta), sin(theta) and theta dot. 
Theta is normalized between -pi and pi.

## Action
Joint effort -2.0 to +2.0
Write a function to discretize the continuous action space of the joint effort.


In [136]:
# define the action space
def create_action_bins(num_action_bins):
    actionbins = np.linspace(-2.0, 2.0, num_action_bins)
    
    return actionbins

# depending on the action, find the according actionbin 
# discretization of the continuous action space
def find_actionbin(action, actionbins):
    idx = (np.abs(actionbins - action)).argmin()

    return idx

## Reward
The reward is defined as
> -(theta^2 + 0.1 x theta_dt^2 + 0.001 x action^2)

What is the lowest expected cost? And what is the highest cost?

-(pi^2 + 0.1 x 8^2 + 0.001 x 2^2) = -16.2736044

-(0^2 + 0.1 x 0^2 + 0.001 x 0^2) = 0

From this reward function, what is the goal of the agent?
In essence, the goal is to remain at zero angle (vertical), with the least rotational velocity, and the least effort.

For a hint have a look at the [wiki](https://github.com/openai/gym/wiki).

In [137]:
def train_model(memory, gamma=0.9):
    for state, action, reward, state_new in memory:
        
        # flatten state to make it compatible to our neural network
        flat_state_new = np.reshape(state_new, [1, 3])
        flat_state = np.reshape(state, [1, 3])

        # determine estimated reward given state s' after action a, 
        # combination of observed and predicted exploited reward.
        target = reward + gamma * np.amax(model.predict(flat_state_new))
        # determine current expected agent rewards
        targetfull = model.predict(flat_state)
        
        # update current expected rewards with the emulated prediced reward
        targetfull[0][action] = target
        
        # Fit model based on emulation and prediction
        model.fit(flat_state, targetfull, epochs=1, verbose=0)

## Deep Q Model

As a reminder, this is our Q function.
> Q(s, a) = r + gamma max_a'(Q(s, a'))

The input of our neural network, our generalizable Q-matrix, will be the observation or the state of the pendulum. 
and the output will be the estimate of the reward taking the action a'. Gamma is the discount factor of the predicted reward in our next state. r is the reward 

For our first network we will implement a DQN with keras:

- Layer with 128 ReLU units
- Layer with 64 ReLU units
- 3 inputs and one output per action bin with linear activation function
- Adam optimizer with learning rate 0.0002, beta_1 0.9 and beta_2 0.999
- Loss mean squared error

In [138]:
# Define the Deep-Q-Network in keras

def build_model(num_output_nodes):
    model = Sequential()

    model.add(Dense(128, input_shape=(3,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_output_nodes, activation='linear'))

    adam = optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999)
    model.compile(loss='mse', optimizer=adam)

    return model

In [145]:
def run_episodes(epsilon, gamma, training_iterations, sequence_iterations):
    
    epsilon_decay = 0.9999
    epsilon_min = 0.02
    steps_per_sequence = 250

    for epoch in range(0, training_iterations // sequence_iterations):
        for sequence_id in range(0, sequence_iterations):
            state = env.reset()
            memory = deque()
            
            total_reward = 0
            
            # Easy implementation of decaying exploration
            if epsilon > epsilon_min:
                epsilon = epsilon * epsilon_decay
            
            for i in range(0, steps_per_sequence):
                    
                # exploration
                if np.random.uniform() < epsilon:
                    action = env.action_space.sample()
                # exploitation
                else:
                    flat_state = np.reshape(state, [1, 3])
                    action = np.amax(model.predict(flat_state))

                # determine action
                actionbin = find_actionbin(action, actionbinslist)
                action = actionbinslist[actionbin]
                action = np.array([action])

                # emulate the action in the simulation and observe the transition 
                # as well as the reward
                observation, reward, done, _ = env.step(action)
                total_reward += reward

                state_new = observation

                # save transitions into
                memory.append((state, actionbin, reward, state_new))

                state = state_new
                
            # train model on the samples memories
            train_model(memory, gamma)
            
            print(epoch , ' epoch', sequence_id, ' sequence. Average reward = ', total_reward / steps_per_sequence, '. epsilon = ', epsilon)

           

## Function for running the policy of our DQN after loading or training


In [146]:
def play_game(rounds):
    state = env.reset()
    totalreward = 0

    for _ in range(0, rounds):
        # Rendering for visualization
        env.render()

        flat_state = np.reshape(state, [1, 3])
        actionbin = np.argmax(model.predict(flat_state))

        action = actionbinslist[actionbin]
        action = np.array([action])

        observation, reward, done, _ = env.step(action)

        totalreward += reward

        state_new = observation
        state = state_new
        
    return totalreward

## Train the DQN


In [ ]:
env = gym.make('Pendulum-v0')


# iterations
training_iterations = 1000
sequence_iterations = 25

# epsilon (setting exploitation vs exploration)
epsilon = 1

# gamma (importance of predicted estimated reward)
gamma = 0.9

# Discretization settings for the action space
num_action_bins = 20
actionbinslist = create_action_bins(num_action_bins)



# Build model
model = build_model(num_action_bins)

run_episodes(epsilon, gamma, training_iterations, sequence_iterations)

   

Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x00000205B5A02438>>
Traceback (most recent call last):
  File "C:\Users\Z624284\AppData\Local\Continuum\anaconda3\envs\juno.tf\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 39, in __del__
    c_api.TF_DeleteStatus(self.status)
AttributeError: 'ScopedTFStatus' object has no attribute 'status'


0  epoch 0  sequence. Average reward =  -6.1693816682624405 . epsilon =  0.9999
0  epoch 1  sequence. Average reward =  -6.103361849396182 . epsilon =  0.9998000100000001
0  epoch 2  sequence. Average reward =  -7.4376313848445115 . epsilon =  0.9997000299990001
0  epoch 3  sequence. Average reward =  -4.804119388389292 . epsilon =  0.9996000599960002
0  epoch 4  sequence. Average reward =  -5.238030700912224 . epsilon =  0.9995000999900007
0  epoch 5  sequence. Average reward =  -6.103151804757043 . epsilon =  0.9994001499800017
0  epoch 6  sequence. Average reward =  -7.2355087385584245 . epsilon =  0.9993002099650037
0  epoch 7  sequence. Average reward =  -6.091206494715181 . epsilon =  0.9992002799440072
0  epoch 8  sequence. Average reward =  -4.50672234640893 . epsilon =  0.9991003599160128
0  epoch 9  sequence. Average reward =  -6.0409351612612 . epsilon =  0.9990004498800211
0  epoch 10  sequence. Average reward =  -6.144442319953128 . epsilon =  0.9989005498350332
0  epoch 1

3  epoch 15  sequence. Average reward =  -5.035032888284185 . epsilon =  0.9909408287818039
3  epoch 16  sequence. Average reward =  -8.774774093637603 . epsilon =  0.9908417346989258
3  epoch 17  sequence. Average reward =  -6.596894512713364 . epsilon =  0.9907426505254558
3  epoch 18  sequence. Average reward =  -8.702508151872454 . epsilon =  0.9906435762604033
3  epoch 19  sequence. Average reward =  -7.5386949864379975 . epsilon =  0.9905445119027773
3  epoch 20  sequence. Average reward =  -5.64687164224414 . epsilon =  0.990445457451587
3  epoch 21  sequence. Average reward =  -6.213091834810061 . epsilon =  0.9903464129058418
3  epoch 22  sequence. Average reward =  -6.269378891987121 . epsilon =  0.9902473782645512
3  epoch 23  sequence. Average reward =  -6.148349646770915 . epsilon =  0.9901483535267248
3  epoch 24  sequence. Average reward =  -6.039287694925221 . epsilon =  0.9900493386913721
4  epoch 0  sequence. Average reward =  -7.8429714472451835 . epsilon =  0.989950

7  epoch 5  sequence. Average reward =  -5.178568457756237 . epsilon =  0.982061932340002
7  epoch 6  sequence. Average reward =  -5.0946314420730605 . epsilon =  0.9819637261467681
7  epoch 7  sequence. Average reward =  -4.635768419783698 . epsilon =  0.9818655297741534
7  epoch 8  sequence. Average reward =  -7.726272129654802 . epsilon =  0.981767343221176
7  epoch 9  sequence. Average reward =  -8.224994474098535 . epsilon =  0.9816691664868539
7  epoch 10  sequence. Average reward =  -4.381272065192495 . epsilon =  0.9815709995702052
7  epoch 11  sequence. Average reward =  -5.703149011562138 . epsilon =  0.9814728424702482
7  epoch 12  sequence. Average reward =  -6.4138739981201365 . epsilon =  0.9813746951860012
7  epoch 13  sequence. Average reward =  -8.234966890354427 . epsilon =  0.9812765577164826
7  epoch 14  sequence. Average reward =  -7.9809076750995285 . epsilon =  0.9811784300607109
7  epoch 15  sequence. Average reward =  -7.362387144720857 . epsilon =  0.981080312

10  epoch 20  sequence. Average reward =  -4.379460374934107 . epsilon =  0.9732625914072018
10  epoch 21  sequence. Average reward =  -6.598131330500858 . epsilon =  0.9731652651480611
10  epoch 22  sequence. Average reward =  -6.064414131594992 . epsilon =  0.9730679486215463
10  epoch 23  sequence. Average reward =  -7.146010381489505 . epsilon =  0.9729706418266841
10  epoch 24  sequence. Average reward =  -7.0212686903085695 . epsilon =  0.9728733447625014
11  epoch 0  sequence. Average reward =  -8.669303633755003 . epsilon =  0.9727760574280252
11  epoch 1  sequence. Average reward =  -6.063678440307088 . epsilon =  0.9726787798222825
11  epoch 2  sequence. Average reward =  -4.803067127447964 . epsilon =  0.9725815119443002
11  epoch 3  sequence. Average reward =  -4.626108819585894 . epsilon =  0.9724842537931058
11  epoch 4  sequence. Average reward =  -7.660444336154984 . epsilon =  0.9723870053677265
11  epoch 5  sequence. Average reward =  -4.977998160688636 . epsilon =  0

...

180  epoch 19  sequence. Average reward =  -5.684602972853031 . epsilon =  0.6363397873245691
180  epoch 20  sequence. Average reward =  -5.332605287725426 . epsilon =  0.6362761533458366
180  epoch 21  sequence. Average reward =  -5.301074163385677 . epsilon =  0.636212525730502
180  epoch 22  sequence. Average reward =  -5.784008144871892 . epsilon =  0.6361489044779289
180  epoch 23  sequence. Average reward =  -7.033185251261092 . epsilon =  0.6360852895874811
180  epoch 24  sequence. Average reward =  -4.874864160818905 . epsilon =  0.6360216810585224
181  epoch 0  sequence. Average reward =  -4.7206083075299 . epsilon =  0.6359580788904166
181  epoch 1  sequence. Average reward =  -5.194299626888136 . epsilon =  0.6358944830825275
181  epoch 2  sequence. Average reward =  -5.080902731338963 . epsilon =  0.6358308936342193
181  epoch 3  sequence. Average reward =  -6.204663149553087 . epsilon =  0.6357673105448559
181  epoch 4  sequence. Average reward =  -8.100248352131523 . epsi

184  epoch 7  sequence. Average reward =  -8.54883793138551 . epsilon =  0.6307642866019273
184  epoch 8  sequence. Average reward =  -5.683754821808133 . epsilon =  0.6307012101732671
184  epoch 9  sequence. Average reward =  -5.632047460505272 . epsilon =  0.6306381400522497
184  epoch 10  sequence. Average reward =  -5.613237430643711 . epsilon =  0.6305750762382445
184  epoch 11  sequence. Average reward =  -5.18396318612355 . epsilon =  0.6305120187306207
184  epoch 12  sequence. Average reward =  -7.012914476374358 . epsilon =  0.6304489675287476
184  epoch 13  sequence. Average reward =  -6.915318647822295 . epsilon =  0.6303859226319947
184  epoch 14  sequence. Average reward =  -6.593985617790862 . epsilon =  0.6303228840397315
184  epoch 15  sequence. Average reward =  -5.213003091919355 . epsilon =  0.6302598517513275
184  epoch 16  sequence. Average reward =  -4.6547032476261325 . epsilon =  0.6301968257661523
184  epoch 17  sequence. Average reward =  -8.074878302289678 . 

187  epoch 20  sequence. Average reward =  -4.976742454122402 . epsilon =  0.6252376374660126
187  epoch 21  sequence. Average reward =  -5.427275783806253 . epsilon =  0.625175113702266
187  epoch 22  sequence. Average reward =  -6.8428701188935035 . epsilon =  0.6251125961908958
187  epoch 23  sequence. Average reward =  -4.905420895361912 . epsilon =  0.6250500849312767
187  epoch 24  sequence. Average reward =  -6.446400547403449 . epsilon =  0.6249875799227835
188  epoch 0  sequence. Average reward =  -5.7240542236657355 . epsilon =  0.6249250811647913
188  epoch 1  sequence. Average reward =  -6.705640927029241 . epsilon =  0.6248625886566748
188  epoch 2  sequence. Average reward =  -8.3964778711239 . epsilon =  0.6248001023978091
188  epoch 3  sequence. Average reward =  -7.049794844259552 . epsilon =  0.6247376223875694
188  epoch 4  sequence. Average reward =  -4.725421993760772 . epsilon =  0.6246751486253306
188  epoch 5  sequence. Average reward =  -7.380105971378851 . eps

191  epoch 8  sequence. Average reward =  -5.122721265983215 . epsilon =  0.6197594118875509
191  epoch 9  sequence. Average reward =  -5.548890488321516 . epsilon =  0.6196974359463622
191  epoch 10  sequence. Average reward =  -5.140224503690388 . epsilon =  0.6196354662027675
191  epoch 11  sequence. Average reward =  -6.074149755339962 . epsilon =  0.6195735026561473
191  epoch 12  sequence. Average reward =  -7.855104598361924 . epsilon =  0.6195115453058817
191  epoch 13  sequence. Average reward =  -5.6671537806399375 . epsilon =  0.6194495941513511
191  epoch 14  sequence. Average reward =  -5.744112256851253 . epsilon =  0.619387649191936
191  epoch 15  sequence. Average reward =  -5.20457819730191 . epsilon =  0.6193257104270168
191  epoch 16  sequence. Average reward =  -7.6533904911864585 . epsilon =  0.6192637778559741
191  epoch 17  sequence. Average reward =  -6.115223379848752 . epsilon =  0.6192018514781885
191  epoch 18  sequence. Average reward =  -5.7050327793502635

194  epoch 21  sequence. Average reward =  -4.743162624342451 . epsilon =  0.614329185587588
194  epoch 22  sequence. Average reward =  -8.182588677565935 . epsilon =  0.6142677526690292
194  epoch 23  sequence. Average reward =  -6.240504382682751 . epsilon =  0.6142063258937623
194  epoch 24  sequence. Average reward =  -4.003567357695198 . epsilon =  0.6141449052611729
195  epoch 0  sequence. Average reward =  -7.948768947893388 . epsilon =  0.6140834907706468
195  epoch 1  sequence. Average reward =  -8.617906958543776 . epsilon =  0.6140220824215697
195  epoch 2  sequence. Average reward =  -5.661984642715342 . epsilon =  0.6139606802133276
195  epoch 3  sequence. Average reward =  -6.101062981658603 . epsilon =  0.6138992841453063
195  epoch 4  sequence. Average reward =  -7.738224318700729 . epsilon =  0.6138378942168917
195  epoch 5  sequence. Average reward =  -5.7556704796606954 . epsilon =  0.6137765104274701
195  epoch 6  sequence. Average reward =  -8.200271497284259 . eps

In [ ]:

# Save model weights

print('saving model')
model.save('pendulum_model_juno_' + str(training_iterations) + '.h5')
print('model saved')


In [148]:
# Evaluate performance on 10 test runs with 100 steps each
trarray = []
rounds = 100
for i in range(10):
    trarray.append(play_game(rounds))
    print(i, ' sequence. Average test reward = ', np.average(trarray)/rounds, 'Average test reward = ', trarray[-1]/rounds)
    

0  sequence. Average test reward =  -3.4512197585919715 Average test reward =  -3.4512197585919715
1  sequence. Average test reward =  -3.009438806659382 Average test reward =  -2.567657854726793
2  sequence. Average test reward =  -2.4452600521959984 Average test reward =  -1.3169025432692325
3  sequence. Average test reward =  -2.1629505577161874 Average test reward =  -1.3160220742767534
4  sequence. Average test reward =  -1.9895910320026298 Average test reward =  -1.2961529291483982
5  sequence. Average test reward =  -1.884748807913826 Average test reward =  -1.3605376874698074
6  sequence. Average test reward =  -1.9707895391840953 Average test reward =  -2.4870339268057107
7  sequence. Average test reward =  -2.048546774529388 Average test reward =  -2.5928474219464364
8  sequence. Average test reward =  -2.2271028859710635 Average test reward =  -3.655551777504468
9  sequence. Average test reward =  -2.136622170053155 Average test reward =  -1.3222957267919802


## Run pretrained model

In case you already trained a model or want to load the pretrained model for sanity checking use the following script (make sure you executed the necessary cells starting with the imports).

- How does the performance change with the amount of trained iterations?
- How can we measure performance?
- Is it sufficient to start the play_game function a single time? 
- How can we make sure, that the evaluation is meaningful?



In [74]:
env = gym.make('Pendulum-v0')

actionbinslist = create_action_bins(20)

# 'pendulum_model_[iterationstrained].h5' 
# iterationstrained: 100, 1000, 10000
model = load_model('pendulum_model_1000.h5')

for i in range(10):
    play_game(rounds=250)
    
env.close()

## Next steps to take it from here

- Implement a skip frame approach
- Experiment with the discretization of the action bins (e.g. advantages and disadvantages of triadisation)
- Experiment with exploration vs exploitation